In [ ]:
system("conda install -y r-mnormt r-rjags")
install.packages("SIBER")

In [ ]:
library(jsonlite)
library(data.table)
library(dplyr)
library(ggplot2)

json_data <- fromJSON("galaxy_inputs/galaxy_inputs.json")

path_file <- json_data$data$path
df        <- fread(path_file)
color     <- json_data$color

if (!is.null(color)) {
  list_color <- strsplit(color, ",")[[1]]
  color_list <- gsub("__pd__", "#", list_color)
}else{color_list <- NULL}
print(color_list)
if(!is.null(json_data$fistXplot)){
  first_list <- trimws(unlist(strsplit(json_data$fistXplot, ","))) 
  first_dpi_img <- as.numeric(first_list[1])
  first_width_img <- as.numeric(first_list[2])
  first_height_img <- as.numeric(first_list[3])
}else{
  first_dpi_img    <- NULL
  first_width_img  <- NULL
  first_height_img <- NULL
}
if(!is.null(json_data$secondXplot)){
  second_list <- trimws(unlist(strsplit(json_data$secondXplot, ",")))
  second_dpi_img <- as.numeric(second_list[1])
  second_width_img <- as.numeric(second_list[2])
  second_height_img <- as.numeric(second_list[3])
}else{
    second_dpi_img    <- NULL
    second_width_img  <- NULL
    second_height_img <- NULL

}

# Supposons que tu as une liste des indices de colonnes (numéros)
col_list_num <- as.integer(trimws(unlist(strsplit(json_data$colXdata, ","))))

# Ensuite, pour chaque indice, tu récupères le nom de la colonne
col_list_names <- names(df)[col_list_num]

# Puis tu assignes les variables avec ces noms de colonnes
GroupID    <- col_list_names[1]
Delta13C   <- col_list_names[2]  # par exemple col 7, récupéré sous forme de nom
PercentC   <- col_list_names[3]
Delta15N   <- col_list_names[4]
PercentN   <- col_list_names[5]

mean_x <- paste0("mean_", Delta13C)
mean_y <- paste0("mean_", Delta15N)
sd_x <- paste0("sd_", Delta13C)
sd_y <- paste0("sd_", Delta15N)

if (!is.null(json_data$separator)) {
  if (grepl(":", json_data$separator)) {
    parts <- strsplit(json_data$separator, ":")[[1]]
    grouping_factor_num <- as.integer(parts[1])
    group_filter_value <- parts[2]
  } else {
    grouping_factor_num <- as.integer(json_data$separator)
    group_filter_value <- NULL
  }
  
  grouping_factor_name <- names(df)[grouping_factor_num]
}else {
  grouping_factor_name <- NULL
}

if (!is.null(grouping_factor_name)) {
  if (!is.null(group_filter_value)) {
    if (!is.null(color_list) && length(color_list) >= 1) {
      palette <- setNames(color_list[1], group_filter_value)
    } else {
      palette <- setNames(grDevices::rainbow(1), group_filter_value)
    }
  } else {
    # Aucun filtre : on prend tous les niveaux
    all_group_levels <- unique(df[[grouping_factor_name]])
    if (is.null(color_list) || length(color_list) < length(all_group_levels)) {
      palette <- grDevices::rainbow(length(all_group_levels))
    } else {
      palette <- color_list
    }
    names(palette) <- all_group_levels
  }
}



if (!is.null(grouping_factor_name)) {
  
  # Si un filtre de valeur est spécifié, on filtre d'abord le dataframe
  if (!is.null(group_filter_value)) {
    df <- df[df[[grouping_factor_name]] == group_filter_value, ]
  }

  total_means <- df %>%
    group_by(!!sym(grouping_factor_name), !!sym(GroupID)) %>%
    summarise(
      !!mean_x := mean(!!sym(Delta13C), na.rm = TRUE),
      !!mean_y := mean(!!sym(Delta15N), na.rm = TRUE),
      !!sd_x := sd(!!sym(Delta13C), na.rm = TRUE),
      !!sd_y := sd(!!sym(Delta15N), na.rm = TRUE),
      n = n(),
      .groups = "drop"
    )

  hulls_comm <- total_means %>%
    group_by(!!sym(grouping_factor_name)) %>%
    slice(chull(!!sym(mean_x), !!sym(mean_y)))
} else {
  total_means <- df %>%
    group_by(!!sym(GroupID)) %>%
    summarise(
        !!mean_x := mean(!!sym(Delta13C), na.rm = TRUE),
        !!mean_y := mean(!!sym(Delta15N), na.rm = TRUE),
        !!sd_x := sd(!!sym(Delta13C), na.rm = TRUE),
        !!sd_y := sd(!!sym(Delta15N), na.rm = TRUE),
        n = n(),
        .groups = "drop"
    )
}

In [ ]:
p <- ggplot(total_means, aes(x = !!sym(mean_x), y = !!sym(mean_y))) +
  geom_point(size = 1) +
  geom_text(aes(label = paste0(!!sym(GroupID), " (n = ", n, ")")), vjust = -1.5, hjust = 0.5, size = 1) +
  labs(x = Delta13C, y = Delta15N) +
  geom_errorbar(aes(ymin = !!sym(mean_y) - !!sym(sd_y), ymax = !!sym(mean_y) + !!sym(sd_y)),width = 0.1,linewidth =0.3) + 
  geom_errorbarh(aes(xmin = !!sym(mean_x) - !!sym(sd_x), xmax = !!sym(mean_x) + !!sym(sd_x)), height = 0.1,linewidth =0.3) + 
  theme_minimal() +
  theme(legend.title = element_blank())


args <- list(
  filename = "outputs/collection/biplot.png",
  plot = p
)
if (!is.null(first_dpi_img)) {
  args$dpi <- first_dpi_img
}
if (!is.null(first_width_img)) {
  args$width <- first_width_img
  args$units <- "px"
}
if (!is.null(first_height_img)) {
  args$height <- first_height_img
  args$units <- "px"
}
try(do.call(ggsave, args), silent = TRUE)
if (!file.exists("outputs/collection/biplot.png")) {
  ggsave("outputs/collection/biplot.png", plot = p)
}


In [ ]:
if (!is.null(grouping_factor_name)) {

  hull_plot <- ggplot(hulls_comm,  aes(x = !!sym(mean_x), y = !!sym(mean_y),
    colour = factor(!!sym(grouping_factor_name)),
    fill = factor(!!sym(grouping_factor_name))
  )) +
    geom_point(size = 2) +
    ylab(paste0(Delta15N, " (%)")) +
    xlab(paste0(Delta13C, " (%)")) +
    scale_colour_manual(name = grouping_factor_name, values = palette) +
    scale_fill_manual(name = grouping_factor_name, values = palette) +
    geom_polygon(alpha = 0.3) +
    theme_bw()

  print(hull_plot)
    
  # Sauvegarde hull plot
    args <- list(
      filename = "outputs/collection/biplot_hull.png",
      plot = hull_plot
    )
    
    if (!is.null(second_dpi_img)) {
      args$dpi <- second_dpi_img
    }
    
    if (!is.null(second_width_img)) {
      args$width <- second_width_img
      args$units <- "px"
    }
    if (!is.null(second_height_img)) {
      args$height <- second_height_img
      args$units <- "px"
    }
    
    try(do.call(ggsave, args), silent = TRUE)
    
    if (!file.exists("outputs/collection/biplot_hull.png")) {
      ggsave("outputs/collection/biplot_hull.png", plot = hull_plot)
    }
}




In [ ]:
if (!is.null(grouping_factor_name)) {
  library(SIBER)
  
  # Construire le dataframe au format attendu par SIBER
  # Ici j’assume que :
  # - iso1 correspond à la moyenne de Delta13C (colonne mean_Delta13C)
  # - iso2 correspond à la moyenne de Delta15N (colonne mean_Delta15N)
  # - group correspond au GroupID (ex: espèces)
  # - community correspond au facteur de regroupement (ex: écozone)
  
  data_pour_siber <- data.frame(
    iso1 = total_means[[paste0("mean_", Delta13C)]],
    iso2 = total_means[[paste0("mean_", Delta15N)]],
    group = total_means[[GroupID]],
    community = total_means[[grouping_factor_name]]
  )
  
  # Créer un objet SIBER au format attendu
  siber.full <- createSiberObject(data_pour_siber)
  
  # Calculer les métriques Layman pour chaque communauté
  Layman.full.calc <- communityMetricsML(siber.full)
  
  # Afficher les résultats
  print(Layman.full.calc)
  write.csv(Layman.full.calc, "outputs/collection/Layman.full.calc.csv", row.names = FALSE)
}

